# Hidden Markov Models

___

States - L, H 

Observations - A, C, T, G

___

Start Probabilities:

|  L  |  H  |
| --- | --- |
| -1 | -1 |

___

Transition Probabilities:

|   | To L | To H |
| --- | --- | --- |
| From L | -0.737| -1.322 |
| From H | -1 | -1 |

___

Observation Probabilities:

|   | A | C | G | T |
| --- | --- | --- | --- | --- |
| L | -1.737 | -2.322 | -2.322 | -1.737 |
| H | -2.322 | -1.737 | -1.737 | -2.322 |

In [7]:
def observation_probability(observation_m, observation_idx, state_idx):
    return observation_m[state_idx][observation_idx]

def starting_probability(start_m, state_idx):
    return start_m[state_idx]

def transition_probability(prev_state_idx, state_idx):
    return transition_m[prev_state_idx][state_idx]
    
def get_maximum(values):
    cur_max = (-float('inf'), None)
    for value in values:
        if value[0] > cur_max[0]:
            cur_max = value
    return cur_max

def get_values_decisions(raw_seq, obs_order, state_order,
              start_m, observation_m, transition_m):
    
    sequence = [x for x in raw_seq]
    
    values = [[] for _ in range(len(state_order))]
    decisions = []
    
    for idx, val in enumerate(sequence):
        sub_decisions = []
        val_idx = obs_order.index(val)
        if idx == 0:
            for s_idx in range(len(state_order)):
                start_proba = starting_probability(start_m, s_idx)
                obs_proba = observation_probability(observation_m, val_idx, s_idx)
                proba = start_proba + obs_proba
                values[s_idx].append(proba)
                sub_decisions.append((proba, -1, s_idx))
        else:
            for s_idx in range(len(state_order)):
                probas = []
                obs_proba = observation_probability(observation_m, val_idx, s_idx)
                for prev_s_idx in range(len(state_order)):
                    last_proba = values[prev_s_idx][idx-1]
                    transition_proba = transition_probability(prev_s_idx, s_idx)
                    proba = last_proba+transition_proba
                    probas.append((proba, prev_s_idx, s_idx))
                most_probable = get_maximum(probas)
                final_proba = obs_proba + most_probable[0]
                values[s_idx].append(round(final_proba, 3))
                sub_decisions.append((final_proba, most_probable[1], most_probable[2]))
        decision = get_maximum(sub_decisions)
        rounded_decision = (round(decision[0], 3), decision[1], decision[2])
        decisions.append(rounded_decision)
    return values, decisions

def get_prob_path(decisions, state_order):
    path = []
    for idx in reversed(range(len(decisions))):
        if idx == len(decisions)-1:
            path.append(state_order[decisions[idx][2]])
        if idx != 0:                
            path.append(state_order[decisions[idx][1]])
    path.reverse()
    return path

In [8]:
start_m = [-1, -1] # [L, H]

transition_m = [[-0.737, -1.322], [-1, -1]] # [From L, From H] with order [To L, To H]

observation_m = [[-1.737, -2.322, -2.322, -1.737], 
               [-2.322, -1.737, -1.737, -2.322]] # [L, H] with order = [A, C, G, T]

state_order = ['L', 'H']

obs_order = ['A', 'C', 'G', 'T']

In [9]:
raw_seq = 'GGCACTGAA'

values, decisions = get_values_decisions(raw_seq, obs_order, state_order, 
                            start_m, observation_m, transition_m)

path = get_prob_path(decisions, state_order)

In [10]:
print(values)
print('DECISION = (PROBABILITY (LN2), STATE OF MAX FOR COLUMN, PREVIOUS STATE)')
print(decisions)
print(path)

[[-3.322, -6.059, -8.796, -10.948, -14.007, -16.481, -19.54, -22.014, -24.488], [-2.737, -5.474, -8.211, -11.533, -14.007, -17.329, -19.54, -22.862, -25.658]]
DECISION = (PROBABILITY (LN2), STATE OF MAX FOR COLUMN, PREVIOUS STATE)
[(-2.737, -1, 1), (-5.474, 1, 1), (-8.211, 1, 1), (-10.948, 1, 0), (-14.007, 0, 1), (-16.481, 0, 0), (-19.54, 0, 0), (-22.014, 0, 0), (-24.488, 0, 0)]
['H', 'H', 'H', 'L', 'L', 'L', 'L', 'L', 'L']


In [11]:
def alphabetical_sort(order, values):
    sorted_order, sorted_values = zip(*sorted(zip(order, values)))
    return sorted_order, sorted_values

def decisions_parser(decisions, state_order):
    n_states = len(state_order)
    parsed_decisions = [[0]*len(decisions) for _ in range(n_states)]
    for i in range(len(decisions)):
        if i != 0:
            parsed_decisions[decisions[i][1]][i-1] = decisions[i-1][0]
            if decisions[i] == decisions[-1]:
                parsed_decisions[decisions[i][2]][i] = decisions[i][0]
    return parsed_decisions
    
def get_lengths(vals, row_labels, col_labels):
    padding = 2
    lengths = []
    max_label = 0
    for label in row_labels:
        len_label = len(label)
        if len_label > max_label:
            max_label = len_label
    lengths.append(max_label + padding)
    for c_idx, label in enumerate(col_labels):
        max_label = -1
        len_label = len(label)
        if len_label > max_label:
            max_label = len_label
        for i in range(len(row_labels)):
            val = vals[i][c_idx]
            len_val = len(str(val))
            if len_val > max_label:
                max_label = len_val
        lengths.append(max_label + padding)
    return lengths

def out_table(vals, row_labels, col_labels):
    lengths = get_lengths(vals, row_labels, col_labels)
    for i in range(len(row_labels)+1):
        for idx, val in enumerate(lengths):
            if i == 0:
                if idx == 0:
                    print('|', end='')
                    print(' '*val, end='')
                else:
                    print('|', end='')
                    print(' ', end='')
                    print(col_labels[idx-1], end='')
                    subtractable = len(col_labels[idx-1]) - 1
                    space_count = val - subtractable - 2
                    print(' '*space_count, end ='')
            else:
                if idx == 0:
                    print('|', end='')
                    print(' ', end='')
                    print(row_labels[i-1], end='')
                    subtractable = len(row_labels[i-1]) - 1
                    space_count = val - subtractable - 2
                    print(' '*space_count, end ='')
                else:
                    print('|', end='')
                    print(' ', end='')
                    print(vals[i-1][idx-1], end='')        
                    subtractable = len(str(vals[i-1][idx-1])) - 1
                    space_count = val - subtractable - 2
                    print(' '*space_count, end ='')
        print('|')
    
def out_sequence(sequence):
    length = len(sequence)
    for i in range(length):
        print(sequence[i], end=' ')
        if i<length-1:
            print('->', end=' ')
    print('')

In [12]:
sequence = [x for x in raw_seq]

print("Observation Sequence:")
out_sequence(sequence)
print('')

print("Values Calculated:")
sorted_order, sorted_values = alphabetical_sort(state_order, values)
out_table(sorted_values, sorted_order, sequence)
print('')

print("Decisions Made:")
parsed_decisions = decisions_parser(decisions, state_order)
sorted_order, sorted_decisions = alphabetical_sort(state_order, parsed_decisions)
out_table(sorted_decisions, sorted_order, sequence)
print('')

print("Most Likely State Sequence:")
out_sequence(path)

Observation Sequence:
G -> G -> C -> A -> C -> T -> G -> A -> A 

Values Calculated:
|   | G      | G      | C      | A       | C       | T       | G      | A       | A       |
| H | -2.737 | -5.474 | -8.211 | -11.533 | -14.007 | -17.329 | -19.54 | -22.862 | -25.658 |
| L | -3.322 | -6.059 | -8.796 | -10.948 | -14.007 | -16.481 | -19.54 | -22.014 | -24.488 |

Decisions Made:
|   | G      | G      | C      | A       | C       | T       | G      | A       | A       |
| H | -2.737 | -5.474 | -8.211 | 0       | 0       | 0       | 0      | 0       | 0       |
| L | 0      | 0      | 0      | -10.948 | -14.007 | -16.481 | -19.54 | -22.014 | -24.488 |

Most Likely State Sequence:
H -> H -> H -> L -> L -> L -> L -> L -> L 
